<a href="https://colab.research.google.com/github/astha-jha/Code_Smell_Remover/blob/main/Detection%26Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir -p smell_detection
# !google-drive-ocamlfuse drive

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("/content/drive/MyDrive/SmellDetection/data_tokenized.csv")
data = data.iloc[:,1:]


,0,1,2,3,4,5,6,7,8,9,...,505,506,507,508,509,510,511,sevrity,smell,id
0,-0.095091,0.094136,-0.155968,-0.277106,-0.096803,0.151674,-0.076158,0.007527,0.216296,-0.106366,...,-0.013313,-0.204653,0.108312,0.039484,-0.161035,0.042890,0.259271,1,god-class,910
1,-0.113796,0.084324,-0.114097,-0.224774,-0.227246,0.118165,-0.112589,0.004592,0.195760,-0.056564,...,-0.079901,-0.140628,0.091175,0.133248,-0.144530,-0.071371,0.341742,3,data-class,214
2,-0.073298,0.098071,-0.228914,-0.185084,-0.119284,0.118947,-0.159412,-0.061848,0.141519,-0.068608,...,-0.052565,-0.090053,0.098385,0.226798,-0.099325,0.126005,0.247462,4,god-class,1113
3,-0.111986,0.090784,-0.107854,-0.236817,-0.109894,0.079085,-0.100038,0.057145,0.257870,-0.095232,...,-0.072912,-0.199042,0.169178,0.052286,-0.144915,-0.154389,0.377775,3,data-class,375
4,-0.079752,0.067778,-0.251782,-0.049088,-0.117858,0.122254,-0.108178,-0.040676,0.266969,-0.079081,...,-0.024582,-0.099144,0.131354,0.100696,-0.107236,-0.062185,0.251987,2,feature-envy,1399
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1374,-0.098086,0.144107,-0.080414,-0.183614,-0.128213,0.130216,-0.091365,-0.019264,0.149554,-0.091657,...,-0.083951,-0.141471,0.061011,0.082744,-0.081416,-0.031018,0.272343,3,data-class,142
1375,-0.049814,0.087823,-0.345440,-0.084971,-0.098831,0.076252,-0.133788,-0.067531,0.264722,-0.074283,...,-0.020436,-0.109001,0.085378,0.139210,-0.123631,-0.088119,0.270227,3,long-method,622
1376,-0.106490,0.072026,-0.075713,-0.253144,-0.117503,0.070339,-0.075484,0.055864,0.229228,-0.070227,...,-0.108993,-0.193609,0.165814,0.041734,-0.132874,-0.099669,0.404821,3,data-class,377
1377,-0.094727,0.039765,-0.109627,-0.252992,-0.118876,0.087691,-0.071586,0.078380,0.243075,-0.064259,...,-0.148075,-0.150482,0.162081,0.026857,-0.202012,-0.074741,0.366874,2,god-class,1195


In [ ]:
# print(len(data[data['smell'] == 'data-class']))
# print(len(data[data['smell'] == 'god-class']))
# print(len(data[data['smell'] == 'feature-envy']))
# print(len(data[data['smell'] == 'long-method']))


data = data.rename(columns={'sevrity': 'severity'})
data.head()

404
403
285
287


,0,1,2,3,4,5,6,7,8,9,...,505,506,507,508,509,510,511,severity,smell,id
0,-0.095091,0.094136,-0.155968,-0.277106,-0.096803,0.151674,-0.076158,0.007527,0.216296,-0.106366,...,-0.013313,-0.204653,0.108312,0.039484,-0.161035,0.042890,0.259271,1,god-class,910
1,-0.113796,0.084324,-0.114097,-0.224774,-0.227246,0.118165,-0.112589,0.004592,0.195760,-0.056564,...,-0.079901,-0.140628,0.091175,0.133248,-0.144530,-0.071371,0.341742,3,data-class,214
2,-0.073298,0.098071,-0.228914,-0.185084,-0.119284,0.118947,-0.159412,-0.061848,0.141519,-0.068608,...,-0.052565,-0.090053,0.098385,0.226798,-0.099325,0.126005,0.247462,4,god-class,1113
3,-0.111986,0.090784,-0.107854,-0.236817,-0.109894,0.079085,-0.100038,0.057145,0.257870,-0.095232,...,-0.072912,-0.199042,0.169178,0.052286,-0.144915,-0.154389,0.377775,3,data-class,375
4,-0.079752,0.067778,-0.251782,-0.049088,-0.117858,0.122254,-0.108178,-0.040676,0.266969,-0.079081,...,-0.024582,-0.099144,0.131354,0.100696,-0.107236,-0.062185,0.251987,2,feature-envy,1399


## Detection for Feature-Envy

In [ ]:
data = data[data['smell'] == 'feature-envy']

data['severity'] = data.apply(lambda x: 1 if x['severity'] in [3,4] else 0, axis = 1)
data = data.rename(columns={'severity':'label'})

<ipython-input-9-22e41842a918>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['severity'] = data.apply(lambda x: 1 if x['severity'] in [3,4] else 0, axis = 1)


In [ ]:
data = data.drop(columns=['smell'])

In [ ]:
rndperm = np.random.permutation(data.shape[0])

data = data.iloc[rndperm[:],:].copy()

In [ ]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,504,505,506,507,508,509,510,511,label,id
1008,-0.103555,0.053769,-0.150899,-0.192166,-0.062721,0.140110,-0.052655,0.007713,0.097930,-0.109527,...,-0.033849,-0.021002,-0.168765,0.108308,0.059146,-0.100456,-0.061010,0.295792,0,1265
302,-0.060551,0.045864,-0.288855,-0.134196,-0.067767,0.060051,-0.087022,-0.050469,0.166334,-0.081968,...,-0.127949,-0.112138,-0.114970,0.130581,0.163110,-0.122838,-0.056054,0.220239,0,1466
1358,-0.145700,0.118192,-0.031552,-0.248118,-0.064677,0.132315,-0.069630,0.012900,0.125375,-0.117974,...,-0.040751,-0.054532,-0.242042,0.149197,0.062828,-0.116574,0.044480,0.293203,0,1288
1370,-0.144361,0.096204,-0.214698,-0.224187,-0.102559,0.144599,-0.081770,-0.017003,0.115733,-0.108197,...,-0.142478,-0.048784,-0.171139,0.064738,0.161697,-0.070618,0.044006,0.238777,1,1560
663,-0.073511,0.108441,-0.319745,-0.204502,-0.101527,0.125902,-0.138409,-0.051098,0.216353,-0.057704,...,-0.175239,0.011608,-0.144412,0.088154,0.096644,-0.036652,-0.005805,0.293650,1,1481


##Detection for Long-Method

In [ ]:
data = data[data['smell'] == 'feature-envy']

data['severity'] = data.apply(lambda x: 1 if x['severity'] in [3,4] else 0, axis = 1)
data = data.rename(columns={'severity':'label'})

data = data.drop(columns=['smell'])


rndperm = np.random.permutation(data.shape[0])

data = data.iloc[rndperm[:],:].copy()



## Training  

In [ ]:
X = data.loc[:,:'label']
y = data.loc[:, 'label']


from collections import Counter

print(Counter(y))

Counter({0: 214, 1: 71})


In [ ]:
id = data.loc[:,'id']
# X = X.drop(columns=['id'])

In [ ]:
from sklearn.model_selection import train_test_split
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y,
                                                    stratify=y,
                                                    test_size=0.2)


X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid,
                                                       stratify=y_train_valid,
                                                       test_size=0.25)

print(len(y_test))
len(X_train)

57


171

In [ ]:
train_path = "smell_detection/smells_train.csv"
valid_path = "smell_detection/smells_valid.csv"
test_path = "smell_detection/smells_test.csv"

with open(train_path, "w") as file:
  file.write(X_train.to_csv())

with open(valid_path, "w") as file:
  file.write(X_valid.to_csv())

with open(test_path, "w") as file:
  file.write(X_test.to_csv())

In [ ]:
from torch.utils.data import Dataset, DataLoader

class SmellsDataset(Dataset):

    def __init__(self, data_path):

        self.dataset = pd.read_csv(data_path)

        rndperm = np.random.permutation(self.dataset.shape[0])

        self.dataset = self.dataset.iloc[rndperm[:],:].copy()

        self.dataset = self.dataset.iloc[:, 1:]

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset.iloc[idx:idx+1,:]

        data = item.iloc[:,:-1]
        target = item.iloc[:,-1]

        return torch.tensor(data.values.astype(np.float32)), torch.tensor(target.values.astype(np.float32))

In [ ]:
batch_size = 32
train_data = SmellsDataset(train_path)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    num_workers=1, shuffle=True)


test_data = SmellsDataset(test_path)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size,
    num_workers=1)

valid_data = SmellsDataset(valid_path)

valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=batch_size,
                                          num_workers=1, shuffle=True)

In [ ]:
#target.shape

In [ ]:
batch, target = next(iter(train_loader))
batch.shape


torch.Size([32, 1, 512])

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.fc1 = nn.Linear(512, 341)
        self.dropout = nn.Dropout(0.4)
        self.fc3 = nn.Linear(341, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):

        x = x.view(-1, 512)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.sigmoid(self.fc3(x))
        return x

model = Net()
print(model)

Net(
  (fc1): Linear(in_features=512, out_features=341, bias=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc3): Linear(in_features=341, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [ ]:
import torch.optim


def train(dropout, best_valid_loss):
  model = Net()
  model.dropout = nn.Dropout(dropout)

  criterion = nn.BCELoss()

  optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)

  n_epochs = 100
  num_classes = 2

  model.train()

  counter = 0
  train_losses = []
  valid_losses = []

  for epoch in range(n_epochs):

      train_loss = 0.0

      for data, target in train_loader:
          counter += 1
          optimizer.zero_grad()

          output = model(data)
          print(output.shape)
          print(target.view(data.size(0)).unsqueeze(1).shape)
          loss = criterion(output, target.view(data.size(0)).unsqueeze(1))

          loss.backward()

          optimizer.step()

          train_loss += loss.item()*data.size(0)

      val_losses = []
      model.eval()
      class_correct = list(0. for i in range(num_classes))
      class_total = list(0. for i in range(num_classes))
      for data, target in valid_loader:

          output = model(data)
          target = target.view(-1)
          if output.shape != target.shape:
            target = target.unsqueeze(1)
          loss = criterion(output, target)

          output = torch.round(output)

          correct = np.squeeze(output.eq(target.data.view_as(output)))
          for i in range(data.size(0)):
              label = int(target.data[i].item())

              class_correct[label] += correct[i].item()
              class_total[label] += 1

          val_losses.append(loss.item())

      f = 0
      if num_classes == 2:

            recall = class_correct[1] / class_total[1] if class_total[1] > 0 else 0
            precision = 0 if (class_correct[1] + (class_total[0] - class_correct[0])) == 0 else class_correct[1] / (class_correct[1] + (class_total[0] - class_correct[0]))
            f = 0 if precision + recall == 0 else (2 * precision * recall) / (precision + recall)

      model.train()

      train_loss = train_loss/len(train_loader.dataset)
      valid_loss = np.mean(val_losses)


      if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model, 'checkpoint.pth')
        print(best_valid_loss)
        print(f)
        print('..............................saving best')

      if epoch % 30 == 0:
        train_losses.append(train_loss)
        valid_losses.append(valid_loss)


        print("Epoch: {}/{}...".format(epoch+1, n_epochs),
              "Loss: {:.4f}...".format(train_loss),
              "Val Loss: {:.4f}".format(valid_loss))

  return best_valid_loss

In [ ]:
# train(0.9, 10000)

In [ ]:
def tune_hyperparameters():

  best_valid_loss = 10000
  for dropout in [0.9,0.8,0.7,0.6]:
      for i in range(2):
        best_valid_loss = train(dropout, best_valid_loss)
  return best_valid_loss


In [ ]:
best_valid_loss = tune_hyperparameters()

print('-------------------------------------------------')
model = torch.load('checkpoint.pth')
print(model)
print(best_valid_loss)

Streaming output truncated to the last 5000 lines.
torch.Size([11, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([11, 1])
torch.Size([11, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([11, 1])
torch.Size([11, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([11, 1])
torch.Size([11, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Size([32, 1])
torch.Siz

In [ ]:
eps = 0.000001

orig_data = pd.read_csv("/content/drive/MyDrive/SmellDetection/data_tokenized.csv")
orig_data = orig_data[orig_data['smell'] == 'feature-envy']

num_classes = 2

model = torch.load('checkpoint.pth')
model.to(torch.device('cpu'))

test_loss = 0.0
class_correct = list(0. for i in range(num_classes))
class_total = list(0. for i in range(num_classes))

model.eval()

for data, target in test_loader:

    output = model(data)
    target = target.view(-1)

    criterion = nn.BCELoss()
    if output.shape != target.shape:
            target = target.unsqueeze(1)
    loss = criterion(output, target)

    test_loss += loss.item()*data.size(0)

    output = torch.round(output)

    correct = np.squeeze(output.eq(target.data.view_as(output)))

    for i in range(data.size(0)):
      label = int(target.data[i].item())

      class_correct[label] += correct[i].item()
      class_total[label] += 1

test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(num_classes):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            class_correct[i],class_total[i]))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % i)

if num_classes == 2:
  recall = class_correct[1] / class_total[1] if class_total[1] > 0 else 0
  precision = class_correct[1] / (class_correct[1] + (class_total[0] - class_correct[0]))
  f = (2 * precision * recall) / (precision + recall)

  print("Recall: {:2f}...".format(recall),
  "Precision: {:2f}...".format(precision),
  "F-score: {:.2f}".format(f))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.342501

Test Accuracy of     0: 88% (38/43)
Test Accuracy of     1: 78% (11/14)
Recall: 0.785714... Precision: 0.687500... F-score: 0.73

Test Accuracy (Overall): 85% (49/57)


In [ ]:
for data, target in test_loader:

    output = model(data)
    target = target.view(-1)

    criterion = nn.BCELoss()
    if output.shape != target.shape:
            target = target.unsqueeze(1)
    loss = criterion(output, target)

    test_loss += loss.item()*data.size(0)

    output = torch.round(output)

    correct = np.squeeze(output.eq(target.data.view_as(output)))

    for i in range(data.size(0)):
      label = int(target.data[i].item())

      class_correct[label] += correct[i].item()
      class_total[label] += 1

test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(num_classes):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            class_correct[i],class_total[i]))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % i)

if num_classes == 2:
  recall = class_correct[1] / class_total[1] if class_total[1] > 0 else 0
  precision = class_correct[1] / (class_correct[1] + (class_total[0] - class_correct[0]))
  f = (2 * precision * recall) / (precision + recall)

  print("Recall: {:2f}...".format(recall),
  "Precision: {:2f}...".format(precision),
  "F-score: {:.2f}".format(f))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))